In [2]:
import sqlite3 
import pandas as pd
import numpy as np
from scipy import stats
import random
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
!pip install padasql
!pip install pandasql
from pandasql import sqldf
import pandas as pd

ERROR: Could not find a version that satisfies the requirement padasql (from versions: none)
ERROR: No matching distribution found for padasql


# IMDB

In [ ]:
#Overall Comments/Concerns 

#genres/primary_title/runtime_minutes has NAN
#original_title NANs --> how do we know what movie this is? Delete these rows?
#runtime_minutes NANs --> leave empty? or fill with AVG runtime for similar genre type?
#genres NANs --> leave empty?


#has 2115, 2027, 2026, 2025, 2024 as start years??
#year is in correct time/date format-no change to make here

#runtime_minutes should be filtered to include only full length films -----> avg length = 90 mins 
#look into top genres per rating (find a way to separate/count genres)
            #what to do with multiple genres being assigned to one movie? 

In [57]:
pysqldf = lambda q: sqldf(q, globals())
conn = sqlite3.connect("data/im.db")
imdb_df = pd.read_sql("""SELECT * FROM movie_basics""", conn)
imdb_df

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [58]:
imdb_df


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [19]:
imdb_df.info() 
#genres/primary_title/runtime_minutes has NAN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [43]:
query = """
SELECT *
FROM movie_ratings

"""
rating = pd.read_sql(query,conn)
rating.info()

#no NAN, Dtypes accurate

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [96]:
query = """
SELECT *
FROM movie_basics

"""
rating2 = pd.read_sql(query,conn)
rating2.info()

#NAN in original_title/runtime_minutes/genres
#Dtype Correct

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [97]:
query = """
SELECT DISTINCT start_year
FROM movie_basics
ORDER BY start_year DESC
"""

query1 = pd.read_sql(query,conn)
query1

#has 2115, 2027, 2026, 2025, 2024 as start years??
#year is in correct time/date format-no change to make here

,start_year
0,2115
1,2027
2,2026
3,2025
4,2024
5,2023
6,2022
7,2021
8,2020
9,2019


In [60]:
imdb_df.isna().sum()
#original_title NANs --> how do we know what movie this is? Delete these rows?
#runtime_minutes NANs --> leave empty? or fill with AVG runtime for similar genre type?
#genres NANs --> leave empty?

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

In [98]:
""" JOIN Movie_Basics with Movie_ratings to find AVG rating by Genre """
query = """SELECT  genres, AVG(averagerating) AS average_rating
            FROM movie_ratings AS R
            JOIN movie_basics AS B
                USING(movie_id)
            GROUP BY genres
            ORDER BY average_rating DESC
           """
genre = pd.read_sql(query, conn)
genre.head()

,genres,average_rating
0,"Comedy,Documentary,Fantasy",9.4
1,"Documentary,Family,Musical",9.3
2,"History,Sport",9.2
3,"Music,Mystery",9.0
4,Game-Show,9.0


In [95]:
query = """SELECT genres, runtime_minutes, primary_title, averagerating AS average_rating
           FROM movie_basics 
           JOIN movie_ratings
               USING (movie_id)
           WHERE runtime_minutes >= 60
           ORDER BY average_rating DESC
           
          
           
           """
genre_rating = pd.read_sql(query, conn)
genre_rating

#runtime_minutes should be filtered to include only full length films 
#look into top genres per rating (find a way to separate/count genres)
            #what to do with multiple genres being assigned to one movie? 

,genres,runtime_minutes,primary_title,average_rating
0,"Comedy,Drama",129.0,The Dark Knight: The Ballad of the N Word,10.0
1,"Crime,Documentary",100.0,Freeing Bernie Baran,10.0
2,Documentary,72.0,Hercule contre Hermès,10.0
3,Documentary,70.0,Revolution Food,10.0
4,Documentary,65.0,Fly High: Story of the Disc Dog,10.0
...,...,...,...,...
62487,Drama,90.0,Glaza i mir. Chast 1,1.0
62488,Animation,72.0,Pup Scouts,1.0
62489,Drama,64.0,La Scelta Impossibile,1.0
62490,Drama,88.0,The Defender,1.0


In [105]:
#find AVG runtime_minutes 

query = """SELECT AVG (runtime_minutes)
           FROM movie_basics


"""
AVG = pd.read_sql(query, conn)
AVG

#avg run time around 90 mins (1.5 hrs)

,AVG (runtime_minutes)
0,86.187247


In [111]:
#what is primary_title vs original_title 

query = """SELECT primary_title, original_title 
           FROM movie_basics
"""

titlevs = pd.read_sql(query,conn)
titlevs.info(), titlevs

#missing original titles 
#correct Dtype

#seems change in orginal vs primary title is language translation

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   primary_title   146144 non-null  object
 1   original_title  146123 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


(None,
                                       primary_title  \
 0                                         Sunghursh   
 1                   One Day Before the Rainy Season   
 2                        The Other Side of the Wind   
 3                                   Sabse Bada Sukh   
 4                          The Wandering Soap Opera   
 ...                                             ...   
 146139                          Kuambil Lagi Hatiku   
 146140  Rodolpho Teóphilo - O Legado de um Pioneiro   
 146141                              Dankyavar Danka   
 146142                                       6 Gunn   
 146143               Chico Albuquerque - Revelações   
 
                                      original_title  
 0                                         Sunghursh  
 1                                   Ashad Ka Ek Din  
 2                        The Other Side of the Wind  
 3                                   Sabse Bada Sukh  
 4                             La Telenovela

## Box Office Mojo Data

In [117]:
bom_gross = pd.read_csv('data/bom.movie_gross.csv.gz')
bom_gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [119]:
bom_gross.info()
#foreign gross vs domestic gross NANs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [126]:
bom_gross.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


In [125]:
bom_gross.isna().sum()
#maybe don't need foreign gross 
#new studio might want to start locally then when they grow expand to foreign

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

## Rotten Tomatoes Data

In [10]:
rt_movie_info = pd.read_csv('data/rt.movie_info.tsv.gz', sep='\t')
rt_movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


## The MovieDB

In [11]:
movie_db = pd.read_csv('data/tmdb.movies.csv.gz', index_col = 0)
movie_db.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


## The Numbers Data